# Testing Dataset Creation

This notebook creates a simple testing dataset with only Date and Balance columns, including all days in the time series.
The process follows the same initial preprocessing steps as in f_data_preprocessing.ipynb but focuses only on creating a clean Date-Balance dataset.

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import openpyxl
from datetime import datetime

In [2]:
# Load the test dataset
df_test = pd.read_excel("shihara_test_30days_version2.xlsx")

print("Original Test Data Info:")
print(f"Shape: {df_test.shape}")
print(f"Columns: {df_test.columns.tolist()}")
print("\nFirst 5 rows:")
print(df_test.head())

Original Test Data Info:
Shape: (15, 8)
Columns: ['Date', 'Discription', 'Payments', 'Receipts', 'Balance', 'cleaned_particulars', 'Category', 'Cluster']

First 5 rows:
        Date                   Discription  Payments  Receipts    Balance  \
0 2024-12-04  PURCHASE PIYARA FASHION (PVT    4140.0       NaN  100898.04   
1 2024-12-06   WITHDRAWAL LAUGFS-MORATU RM   60005.0       NaN   75893.04   
2 2024-12-09       IB CEFT CHGS B jayantha      30.0       NaN   50863.04   
3 2024-12-09                      wasantha   25000.0       NaN   50893.04   
4 2024-12-11  PURCHASE CARAVAN FRESH-MORAT    2100.0       NaN   48763.04   

            cleaned_particulars                     Category  Cluster  
0  purchase piyara fashion (pvt         Clothing and Apparel       16  
1   withdrawal laugfs-moratu rm   WITHDRAWAL LAUGFSMORATU RM        4  
2       ib ceft chgs b jayantha  IB CEFT CHGS T V F FERNANDO        1  
3                      wasantha                     WASANTHA        3  
4  purch

## Step 1: Extract Date and Balance Columns

Extract only Date and Balance columns from the test dataset, following the same process as the training data preprocessing.

In [3]:
# Extract Date and Balance columns only
df_test_balance = df_test[["Date", "Balance"]].copy()

# Remove the first row (often contains headers or metadata)
df_test_balance = df_test_balance.drop(0)

# Convert Date column to datetime
df_test_balance['Date'] = pd.to_datetime(df_test_balance['Date'])

print("Extracted Data Info:")
print(f"Shape after extraction: {df_test_balance.shape}")
print("\nFirst 5 rows after cleaning:")
print(df_test_balance.head())
print("\nLast 5 rows:")
print(df_test_balance.tail())

Extracted Data Info:
Shape after extraction: (14, 2)

First 5 rows after cleaning:
        Date   Balance
1 2024-12-06  75893.04
2 2024-12-09  50863.04
3 2024-12-09  50893.04
4 2024-12-11  48763.04
5 2024-12-16  70063.04

Last 5 rows:
         Date   Balance
10 2024-12-20  81573.04
11 2024-12-23  81373.04
12 2024-12-24   1373.04
13 2024-12-31  31506.21
14 2025-01-02  30709.21


In [4]:
# Clean and convert Balance column to numeric
print("Balance column before conversion:")
print(f"Data type: {df_test_balance['Balance'].dtype}")
print(f"Sample values: {df_test_balance['Balance'].head()}")

# Remove any non-numeric values and convert to float
df_test_balance['Balance'] = pd.to_numeric(df_test_balance['Balance'], errors='coerce')

# Check for NaN values (were non-numeric)
nan_count = df_test_balance['Balance'].isna().sum()
print(f"\nNumber of NaN values after conversion: {nan_count}")

# Drop rows where Balance is NaN
if nan_count > 0:
    df_test_balance = df_test_balance.dropna(subset=['Balance'])
    print(f"Dropped {nan_count} rows with NaN values")

print(f"\nFinal shape after cleaning: {df_test_balance.shape}")
print(f"Balance column data type: {df_test_balance['Balance'].dtype}")
print(f"Balance range: {df_test_balance['Balance'].min():,.2f} to {df_test_balance['Balance'].max():,.2f}")

Balance column before conversion:
Data type: float64
Sample values: 1    75893.04
2    50863.04
3    50893.04
4    48763.04
5    70063.04
Name: Balance, dtype: float64

Number of NaN values after conversion: 0

Final shape after cleaning: (14, 2)
Balance column data type: float64
Balance range: 1,373.04 to 88,263.04


## Step 2: Create Complete Time Series

Group by date, sort, and create a continuous time series that includes all days (filling missing dates).

In [5]:
# Group by date and take the last entry for each date, then sort
df_test_balance = df_test_balance.groupby('Date', as_index=False).last()
df_test_balance = df_test_balance.sort_values('Date')

print(f"After grouping by date: {df_test_balance.shape}")
print("\nDate range:")
print(f"Start date: {df_test_balance['Date'].min()}")
print(f"End date: {df_test_balance['Date'].max()}")
print(f"Number of unique dates: {df_test_balance['Date'].nunique()}")

# Set Date as index for time series operations
df_test_balance = df_test_balance.set_index('Date')
print(f"\nData with Date as index:")
print(df_test_balance.head())

After grouping by date: (12, 2)

Date range:
Start date: 2024-12-06 00:00:00
End date: 2025-01-02 00:00:00
Number of unique dates: 12

Data with Date as index:
             Balance
Date                
2024-12-06  75893.04
2024-12-09  50893.04
2024-12-11  48763.04
2024-12-16  70063.04
2024-12-17  88263.04


In [7]:
# Create full date range and reindex to fill missing dates
full_index = pd.date_range(start=df_test_balance.index.min(), end=df_test_balance.index.max(), freq='D')
print(f"Full date range length: {len(full_index)}")
print(f"Original data length: {len(df_test_balance)}")
print(f"Missing dates to be filled: {len(full_index) - len(df_test_balance)}")

# Reindex with full date range
df_test_balance = df_test_balance.reindex(full_index)

# Forward fill missing balance values (carry forward the last known balance)
df_test_balance['Balance'] = df_test_balance['Balance'].ffill()

# Reset index to make Date a column again
df_test_balance = df_test_balance.reset_index().rename(columns={'index': 'Date'})

print(f"\nAfter creating continuous time series: {df_test_balance.shape}")
print(f"Any missing values: {df_test_balance['Balance'].isna().sum()}")
print("\nSample data:")
print(df_test_balance.head(10))

Full date range length: 28
Original data length: 12
Missing dates to be filled: 16

After creating continuous time series: (28, 2)
Any missing values: 0

Sample data:
        Date   Balance
0 2024-12-06  75893.04
1 2024-12-07  75893.04
2 2024-12-08  75893.04
3 2024-12-09  50893.04
4 2024-12-10  50893.04
5 2024-12-11  48763.04
6 2024-12-12  48763.04
7 2024-12-13  48763.04
8 2024-12-14  48763.04
9 2024-12-15  48763.04


## Step 3: Final Testing Dataset Summary and Export

Review the final testing dataset and save it to files.

In [8]:
# Final dataset summary
print("=" * 60)
print("FINAL TESTING DATASET SUMMARY")
print("=" * 60)

print(f"Dataset shape: {df_test_balance.shape}")
print(f"Date range: {df_test_balance['Date'].min()} to {df_test_balance['Date'].max()}")
print(f"Total days: {len(df_test_balance)}")
print(f"Columns: {df_test_balance.columns.tolist()}")

print("\nBalance Statistics:")
print(f"Minimum balance: {df_test_balance['Balance'].min():,.2f}")
print(f"Maximum balance: {df_test_balance['Balance'].max():,.2f}")
print(f"Mean balance: {df_test_balance['Balance'].mean():,.2f}")
print(f"Standard deviation: {df_test_balance['Balance'].std():,.2f}")

print("\nFirst 5 rows of final testing dataset:")
print(df_test_balance.head())

print("\nLast 5 rows of final testing dataset:")
print(df_test_balance.tail())

FINAL TESTING DATASET SUMMARY
Dataset shape: (28, 2)
Date range: 2024-12-06 00:00:00 to 2025-01-02 00:00:00
Total days: 28
Columns: ['Date', 'Balance']

Balance Statistics:
Minimum balance: 1,373.04
Maximum balance: 88,263.04
Mean balance: 47,584.56
Standard deviation: 32,120.38

First 5 rows of final testing dataset:
        Date   Balance
0 2024-12-06  75893.04
1 2024-12-07  75893.04
2 2024-12-08  75893.04
3 2024-12-09  50893.04
4 2024-12-10  50893.04

Last 5 rows of final testing dataset:
         Date   Balance
23 2024-12-29   1373.04
24 2024-12-30   1373.04
25 2024-12-31  31506.21
26 2025-01-01  31506.21
27 2025-01-02  30709.21


In [9]:
df_test_balance.head(30)

,Date,Balance
0,2024-12-06,75893.04
1,2024-12-07,75893.04
2,2024-12-08,75893.04
3,2024-12-09,50893.04
4,2024-12-10,50893.04
5,2024-12-11,48763.04
6,2024-12-12,48763.04
7,2024-12-13,48763.04
8,2024-12-14,48763.04
9,2024-12-15,48763.04


In [10]:
# Save the testing dataset
test_output_excel_path = 'processed_test_dataset.xlsx'
test_output_csv_path = 'processed_test_dataset.csv'

# Save to Excel and CSV
df_test_balance.to_excel(test_output_excel_path, index=False)
df_test_balance.to_csv(test_output_csv_path, index=False)

print(f"✓ Testing dataset saved to:")
print(f"  Excel: {test_output_excel_path}")
print(f"  CSV: {test_output_csv_path}")

print(f"\n✓ Testing dataset creation completed successfully! 🎉")
print(f"  - Simple dataset with only Date and Balance columns")
print(f"  - All days included in the time series")
print(f"  - Missing dates filled with forward-filled balance values")
print(f"  - Ready for forecasting and testing purposes")

✓ Testing dataset saved to:
  Excel: processed_test_dataset.xlsx
  CSV: processed_test_dataset.csv

✓ Testing dataset creation completed successfully! 🎉
  - Simple dataset with only Date and Balance columns
  - All days included in the time series
  - Missing dates filled with forward-filled balance values
  - Ready for forecasting and testing purposes
